In [1]:
import re
sy_poly = set()
sy_en_poly = set()
ploy_file = '/mnt/cfs/SPEECH/hupeng/git_loc_workspace/tal_frontend_service/tal_frontend/frontend/g2p_pp/POLYPHONIC_CHARS_704.txt'
with open(ploy_file,'r') as fin :
    for line in fin:
        word = line.strip().split('\t')[0]
        if re.match(r'[\u4e00-\u9fff]', word):
            word = ord(word)
            sy_poly.add(word)
        else:
            sy_en_poly.add(word)

In [2]:
import json
xz = {}
with open(r'/mnt/cfs/SPEECH/hupeng/git_loc_workspace/tal_frontend_service/tal_frontend/frontend/g2p/phonemes/others_map.json', 'r', encoding='utf8') as fin:
    fin = fin.read()
    fin = json.loads(fin)
    for word, ph in fin.items():
        if re.match(r'[\u4e00-\u9fff]', word):
            word = ord(word)
            xz[word] = ph
        elif re.search(r'[a-z]',word):
            word = word.lower()
            word = re.sub(r'[^a-z]','',word)
            xz[word] = ph
        else:
            word = ord(word)
            xz[word] = ph

In [23]:
mono_file = r'/mnt/cfs/SPEECH/hupeng/git_loc_workspace/tal_frontend_service/tal_frontend/frontend/g2p/phonemes/cn/cn_monophone_ori.txt'
lh_poly_file = r'/mnt/cfs/SPEECH/hupeng/git_loc_workspace/tal_frontend_service/tal_frontend/frontend/g2p/phonemes/cn/cn_ployphone.txt'

mono_dict = {}

with open(mono_file, 'r', encoding='utf8') as fin:
    for line in fin:
        code, _, py = line.strip().split('\t', maxsplit=2)
        mono_dict[code] = py

with open(lh_poly_file, 'r', encoding='utf8') as fin:
    for line in fin:
        line = line.strip().split(',')
        line = [i for i in line if '5' not in i]
        word = line[0]
        code = ord(word)
        
        try:
            py = line[1]
        except:
            print(line)
        if not mono_dict.get(code):
            mono_dict[code] = py
mono_dict[33062] = 'te5'
for key, _ in mono_dict.items():
    if xz.get(key):
        mono_dict[key] = xz[key]

['脦']


In [3]:
import re
mono_en_file = r'/mnt/cfs/SPEECH/hupeng/git_loc_workspace/tal_frontend_service/tal_frontend/frontend/g2p/phonemes/en/en_monophone_tal.txt'
mono_en_file = r'/mnt/cfs/SPEECH/hupeng/git_loc_workspace/tal_frontend_service/tal_frontend/frontend/g2p/phonemes/en/cmudict'
mono_en_dict = {}
with open(mono_en_file, 'r', encoding='utf8') as fin:
    for line in fin:
        #word, pron = line.strip().split('\t', maxsplit=1)
        word, _, pron = line.strip().split(' ', maxsplit=2)
        word = word.lower()
        word = re.sub(r'[^a-z]', '', word)
        # if word not in sy_en_poly and not mono_en_dict.get(word):
        if not mono_en_dict.get(word):
            mono_en_dict[word] = pron
            if xz.get(word):
                mono_en_dict[word] = xz.get(word)

save_file = r'/mnt/cfs/SPEECH/hupeng/git_loc_workspace/tal_frontend_service/tal_frontend/frontend/g2p/bertg2pw/english_dict.list'
with open(save_file, 'w', encoding='utf8') as fo:
    #fin.write('en_monophone = {\n')
    for key, value in mono_en_dict.items():
        fo.write(f'{key}\t{value}\n')
        #fin.write(f'\'{key}\':\'{value}\',\n')
    #fin.write('}')
fo.close()

In [27]:
remove_code = set()
for key, value in mono_dict.items():
    if key in sy_poly:
        remove_code.add(key)
for key in remove_code:
    mono_dict.pop(key)

In [28]:
save_file = r'/mnt/cfs/SPEECH/hupeng/git_loc_workspace/tal_frontend_service/tal_frontend/frontend/g2p/bertg2pw/pinyin_dict.py'
with open(save_file, 'w', encoding='utf8') as fo:
    fo.write('cn_monophone = {\n')
    for key, value in mono_dict.items():
        fo.write(f'{key}:\'{value}\',\n')
    fo.write('}')
fo.close()